In [1]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
# h5open("lisa_arch.hdf5", "w") do f
#     # The file is now open, but it's empty
# end

HllSet{10}()

In [2]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/", hll)

In [3]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row)
    col_uuid = string(uuid4())
    Store._ingest_csv(db, assign, col_uuid; limit=10000, offset=10)
end

Processed column: 8


Processed column: 15


In [ ]:
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

In [ ]:
arch = h5open("lisa_arch.hdf5")

In [ ]:
close(arch)

In [ ]:
file = h5open("lisa_arch.hdf5", "r")
data_out = Dict()
Store.read_datasets(file, data_out, "/b6b833df-2fab-490a-af20-52b62b047626/nodes/")
println(data_out)
# Close the file
close(file)

In [ ]:
Graph.unset_lock!(db, :, :)